In [1]:
import os

song_filepath = os.path.join('inputs', 'audios', 'Romeo Santos - R.I.P..mp3')
song_metadata_filepath = os.path.join('inputs', 'audio_metadata', 'Romeo Santos - R.I.P..txt')
moves_filepath = os.path.join('outputs', 'moves.json')

In [2]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    moves_data = json.load(file)

musicality_map = {}
for move_id, move_data in moves_data.items():
    for index, item in enumerate(move_data):
        if item['musicality'] != '-':
            key = f"{item['count']}-{item['musicality']}"
            if key not in musicality_map:
                musicality_map[key] = []
            musicality_map[key].append({'move_id': move_id, 'index': index})
musicality_map

{'5-#4': [{'move_id': '5db13cd', 'index': 13}],
 '5-*': [{'move_id': '9da1eb8', 'index': 0}],
 '7-*': [{'move_id': '9da1eb8', 'index': 2}],
 '5-*2': [{'move_id': '5d80c9a', 'index': 13}],
 '7-*2': [{'move_id': '5d80c9a', 'index': 15}]}

In [10]:
with open(song_metadata_filepath, "r", encoding="utf-8") as f:
    musical_notation = f.read()
    # split() without arguments splits on any whitespace (spaces, tabs, newlines)
    musical_tokens = musical_notation.split()
num_tokens = len(musical_tokens)
num_tokens

32

In [12]:
from BeatNet.BeatNet import BeatNet
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)
beats = estimator.process(song_filepath)
beats

C:\Users\user\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\user\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\user\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


array([[ 0.36,  2.  ],
       [ 0.82,  3.  ],
       [ 1.28,  4.  ],
       [ 1.74,  1.  ],
       [ 2.2 ,  2.  ],
       [ 2.68,  3.  ],
       [ 3.12,  4.  ],
       [ 3.6 ,  1.  ],
       [ 4.06,  2.  ],
       [ 4.52,  3.  ],
       [ 4.98,  4.  ],
       [ 5.44,  1.  ],
       [ 5.9 ,  2.  ],
       [ 6.36,  3.  ],
       [ 6.8 ,  4.  ],
       [ 7.3 ,  1.  ],
       [ 7.76,  2.  ],
       [ 8.2 ,  3.  ],
       [ 8.66,  4.  ],
       [ 9.14,  1.  ],
       [ 9.6 ,  2.  ],
       [10.06,  3.  ],
       [10.52,  4.  ],
       [10.98,  1.  ],
       [11.44,  2.  ],
       [11.9 ,  3.  ],
       [12.36,  4.  ],
       [12.82,  1.  ],
       [13.28,  2.  ],
       [13.74,  3.  ],
       [14.2 ,  4.  ],
       [14.66,  1.  ],
       [15.14,  2.  ],
       [15.6 ,  3.  ],
       [16.06,  4.  ],
       [16.5 ,  1.  ]])

In [13]:
def get_important_beats(beats, num_tokens):
    # Find the first position where index == 1
    start = None
    for i, (time, index) in enumerate(beats):
        if index == 1:
            start = i
            break
    
    if start is None:
        raise ValueError("No row with index == 1 found")
    
    # Slice the subarray of length num_tokens (if enough rows exist)
    end = start + num_tokens
    if end > len(beats):
        raise ValueError("Not enough rows to get num_tokens elements starting at index==1")
    
    return beats[start:end]

important_beats = get_important_beats(beats, num_tokens)
important_beats

array([[ 1.74,  1.  ],
       [ 2.2 ,  2.  ],
       [ 2.68,  3.  ],
       [ 3.12,  4.  ],
       [ 3.6 ,  1.  ],
       [ 4.06,  2.  ],
       [ 4.52,  3.  ],
       [ 4.98,  4.  ],
       [ 5.44,  1.  ],
       [ 5.9 ,  2.  ],
       [ 6.36,  3.  ],
       [ 6.8 ,  4.  ],
       [ 7.3 ,  1.  ],
       [ 7.76,  2.  ],
       [ 8.2 ,  3.  ],
       [ 8.66,  4.  ],
       [ 9.14,  1.  ],
       [ 9.6 ,  2.  ],
       [10.06,  3.  ],
       [10.52,  4.  ],
       [10.98,  1.  ],
       [11.44,  2.  ],
       [11.9 ,  3.  ],
       [12.36,  4.  ],
       [12.82,  1.  ],
       [13.28,  2.  ],
       [13.74,  3.  ],
       [14.2 ,  4.  ],
       [14.66,  1.  ],
       [15.14,  2.  ],
       [15.6 ,  3.  ],
       [16.06,  4.  ]])

In [14]:
import numpy as np
beats_array = [(x % 8) + 1 for x in range(num_tokens)]
selected_beats = np.column_stack((important_beats, musical_tokens, beats_array, [f'{beats_array[index]}-{musical_tokens[index]}' for  index in range(num_tokens)])).tolist()
selected_beats

[['1.74', '1.0', '-', '1', '1--'],
 ['2.2', '2.0', '-', '2', '2--'],
 ['2.68', '3.0', '-', '3', '3--'],
 ['3.12', '4.0', '-', '4', '4--'],
 ['3.6', '1.0', '-', '5', '5--'],
 ['4.06', '2.0', '-', '6', '6--'],
 ['4.52', '3.0', '-', '7', '7--'],
 ['4.98', '4.0', '-', '8', '8--'],
 ['5.44', '1.0', '-', '1', '1--'],
 ['5.9', '2.0', '-', '2', '2--'],
 ['6.36', '3.0', '-', '3', '3--'],
 ['6.8', '4.0', '-', '4', '4--'],
 ['7.3', '1.0', '*', '5', '5-*'],
 ['7.76', '2.0', '-', '6', '6--'],
 ['8.2', '3.0', '-', '7', '7--'],
 ['8.66', '4.0', '-', '8', '8--'],
 ['9.14', '1.0', '-', '1', '1--'],
 ['9.6', '2.0', '-', '2', '2--'],
 ['10.06', '3.0', '-', '3', '3--'],
 ['10.52', '4.0', '-', '4', '4--'],
 ['10.98', '1.0', '-', '5', '5--'],
 ['11.44', '2.0', '-', '6', '6--'],
 ['11.9', '3.0', '-', '7', '7--'],
 ['12.36', '4.0', '-', '8', '8--'],
 ['12.82', '1.0', '-', '1', '1--'],
 ['13.28', '2.0', '-', '2', '2--'],
 ['13.74', '3.0', '-', '3', '3--'],
 ['14.2', '4.0', '-', '4', '4--'],
 ['14.66', '1.0', '

In [18]:
import random

last_move_index = 0
choreography = []
for index, (song_time, _, _, beat_index, musicality_token) in enumerate(selected_beats):
    if musicality_token in musicality_map:
        random_move = random.choice(musicality_map[musicality_token])
        move_index = random_move['index']

        '''
        print(index)
        print(last_move_index)
        print(random_move['index'])
        print('----------------')
        '''
        # TODO: start the video in a smooth way finding similar poses
        index_start_move = min(index - last_move_index, move_index)
        choreography.append({
            'move_id': random_move['move_id'],
            'song_index': index - index_start_move,
            'move_index': move_index - index_start_move
        })
        last_move_index = index

choreography

[{'move_id': '9da1eb8', 'song_index': 12, 'move_index': 0},
 {'move_id': '5db13cd', 'song_index': 15, 'move_index': 0}]

In [20]:
'''
Given the following code, I want to create a video with a song as a background.
The video should have a black screen when no moves are in the choreography,
the choreography looks like
[{'move_id': '9da1eb8', 'song_index': 12, 'move_index': 0},
 {'move_id': '5db13cd', 'song_index': 15, 'move_index': 0}]

Where move_id is a key in a dictionary moves_data. moves_data['5db13cd'] looks like this
[
{'time': 0.18},
{'time': 0.66},
{'time': 1.14},
{'time': 1.62},
{'time': 2.08},
{'time': 2.56},
]
Each row is a beat and their respective time. move_id.mp4 is also a video

I want each move_id.mp4 speed adjusted to the beats of the music (selected_beats). The beats of the music looks like this

[['1.74'],
 ['2.2'],
 ['2.68'],
 ['3.12'],
 ['3.6'],
 ['4.06'],
 ['4.52'],
 ['4.98'],
 ['5.44'],
 ['5.9'],
 ['6.36'],
 ['6.8'],
 ['7.3'],
 ['7.76']
]

where each row is a beat. The column is time.

Also, I want to fill the spaces not taken into account in the choreography with a black screen, but still listening to the song

'''

'''
from moviepy.editor import *

def color_clip(size, duration, fps=25, color=(0,0,0), output='color.mp4'):
    ColorClip(size, color, duration=duration).write_videofile(output, fps=fps)

if __name__ == '__main__':
    size = (200, 100)
    duration = 5
    color_clip(size, duration)
'''

from moviepy import VideoFileClip, vfx, ColorClip

last_song_index = 0
for index, move_metadata in enumerate(choreography):
    move_id = move_metadata['move_id']
    start_song_index = move_metadata['song_index']
    move_index = move_metadata['move_index']

    next_song_index = num_tokens + 2
    if index < len(choreography) - 1:
        next_song_index = choreography[index + 1]['song_index']
    end_move_index = len(moves_data[move_id])
    end_song_index = start_song_index + min(next_song_index - start_song_index, end_move_index - move_index) - 1


    print(last_song_index)
    print(start_song_index)
    print(end_song_index)

IndentationError: expected an indented block after 'if' statement on line 43 (3364008915.py, line 46)

In [50]:
from moviepy import VideoFileClip, AudioFileClip, ColorClip, concatenate_videoclips, vfx
import numpy as np

music_beats = [float(b[0]) for b in selected_beats]

# Song audio
song = AudioFileClip(song_filepath)
song_duration = song.duration


# --- Helper: fit move video to beats ---
def stretch_move(move_id, move_index, song_start_idx, song_end_idx):
    # Number of beats in song slice
    n_song_beats = song_end_idx - song_start_idx

    # Take only that many beats from the move, starting at move_index
    move_beats = [d['time'] for d in moves_data[move_id][move_index:move_index+n_song_beats]]

    if len(move_beats) < 2:
        return None  # not enough beats in the move

    move_duration = move_beats[-1] - move_beats[0]

    # Target beats slice in the song
    target_beats = music_beats[song_start_idx:song_end_idx + 1]
    target_duration = target_beats[-1] - target_beats[0]

    # Load the clip
    clip = VideoFileClip(os.path.join('inputs', 'videos', f"{move_id}.mp4"))

    # Trim to the move duration (relative to first beat)
    clip = clip.subclipped(move_beats[0], move_beats[0] + move_duration)

    # Stretch to fit target beats duration
    clip = clip.with_effects([vfx.MultiplySpeed(factor=move_duration/target_duration)])

    # Force exact duration
    clip = clip.with_duration(target_duration)

    return clip


# --- Build timeline ---
clips = []
current_time = 0

for i, move in enumerate(choreography):
    song_idx = move['song_index']
    next_song_idx = choreography[i+1]['song_index'] if i+1 < len(choreography) else len(music_beats)
    print(song_idx)
    print(next_song_idx)

    # black screen before this move (if gap)
    beat_time = music_beats[song_idx]
    if beat_time > current_time:
        clips.append(ColorClip(size=(640,480), color=(0,0,0), duration=beat_time-current_time))

    # add stretched move
    move_clip = stretch_move(
        move['move_id'],
        move['move_index'],
        song_idx,
        next_song_idx
    )
    clips.append(move_clip)

    current_time = beat_time + move_clip.duration

# black screen until song ends
if current_time < song_duration:
    clips.append(ColorClip(size=(640,480), color=(0,0,0), duration=song_duration-current_time))

# Concatenate all
final_video = concatenate_videoclips(clips, method="compose")

# Set audio
final_video.audio = song

# Export
final_video.write_videofile("final_output.mp4", codec="libx264", audio_codec="aac")

12
15
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.7.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 128, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'video', 'language': None, 'default': True, 'size': [720, 1280], 'bitrate': 5564, 'fps': 30.12, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.19.100 libx264'}}], 'input_number': 0}], 'duration': 8.29, 'bitrate': 5688, 'start': 0.0, 'default_audio_input_number': 0, 'default_audio_stream_number': 0, 'audio_fps': 44100, 'audio_bitrate': 128, 'default_video_input_number': 0, 'default_video_stream_number': 1, 'video_c

MoviePy - Done.
MoviePy - Writing video final_output.mp4



MoviePy - Done !
MoviePy - video ready final_output.mp4
